In [ ]:
import osmnx as ox
from quackosm import convert_geometry_to_geodataframe, convert_pbf_to_geodataframe
from shapely import box
from lonboard import viz
from lonboard.colormap import apply_categorical_cmap
from pooch import retrieve

### Filtering by geometry

#### Simple box

In [ ]:
geometry = box(
    xmin=7.415,
    xmax=7.425,
    ymin=43.727,
    ymax=43.735,
)

result_gdf = convert_geometry_to_geodataframe(geometry_filter=geometry)
result_gdf

In [ ]:
viz([result_gdf, geometry.boundary])

#### Geocoded city

In [ ]:
geometry = ox.geocode_to_gdf("Weymouth, UK")

result_gdf = convert_geometry_to_geodataframe(geometry_filter=geometry.unary_union)
result_gdf

In [ ]:
viz([result_gdf, geometry.boundary])

#### H3 cells

In [ ]:
from srai.h3 import h3_to_geoseries

h3_cells = ["871f1d488ffffff", "891f1d48bbbffff", "8a1f1d48b54ffff"]
h3_geometries = h3_to_geoseries(h3_cells)

result_gdf = convert_geometry_to_geodataframe(geometry_filter=h3_geometries.unary_union)
result_gdf

In [ ]:
viz([result_gdf, h3_geometries.boundary])

### Filtering by OSM tags

#### All of the buildings in Vienna

In [ ]:
geometry = ox.geocode_to_gdf("Vienna, Austria")
tags = {"building": True}

result_gdf = convert_geometry_to_geodataframe(
    geometry_filter=geometry.unary_union, tags_filter=tags, osm_extract_source="BBBike"
)
result_gdf

In [ ]:
viz([result_gdf, geometry.boundary])

#### Streets (highways) without any names

In [ ]:
tags = {"highway": True, "name:*": False}

result_gdf = convert_geometry_to_geodataframe(
    geometry_filter=geometry.unary_union,
    tags_filter=tags,
    keep_all_tags=True,
    osm_extract_source="BBBike",
)
result_gdf

In [ ]:
viz(
    [
        result_gdf[result_gdf.geom_type.isin(("LineString", "MultiLineString"))],
        geometry.boundary,
    ]
)

#### All highway roads ending with `-ary` and amenities without building tag in Estonia 

In [ ]:
estonia_pbf_url = "http://download.geofabrik.de/europe/estonia-latest.osm.pbf"
estonia_pbf_file = retrieve(url=estonia_pbf_url, known_hash=None, path="files")
# Grouped tags filter
tags = {
    "main_roads": {"highway": "*ary"},
    "amenities": {"amenity": True, "building": False},
}

result_gdf = convert_pbf_to_geodataframe(pbf_path=estonia_pbf_file, tags_filter=tags)
result_gdf

In [ ]:
highways_gdf = result_gdf[~result_gdf['main_roads'].isna()]
highways_gdf

In [ ]:
highways_gdf['main_roads'].unique()

In [ ]:
amenities_gdf = result_gdf[~result_gdf['amenities'].isna()]
amenities_gdf

In [ ]:
m = viz([highways_gdf, amenities_gdf])

highway_path_layer = m.layers[0]

highway_path_layer.get_color = apply_categorical_cmap(
    highway_path_layer.table["main_roads"].to_pandas(),
    {
        "highway=primary": [220, 217, 57],
        "highway=secondary": [184, 217, 235],
        "highway=tertiary": [181, 197, 143],
    },
)
highway_path_layer.get_width = (
    highway_path_layer.table["main_roads"]
    .to_pandas()
    .map(
        {
            "highway=primary": 5,
            "highway=secondary": 2,
            "highway=tertiary": 1,
        }
    )
)
highway_path_layer.width_scale = 10
highway_path_layer.width_min_pixels = 0.5

m